In [2]:
using SatelliteToolbox
using Dates

In [39]:
function to_datatime(jd)
    date_tuple = JDtoDate(jd)
    
    DateTime(
        date_tuple[1],
        date_tuple[2],
        date_tuple[3],
        date_tuple[4],
        date_tuple[5],
        round(date_tuple[6]),
    )
end

function TEME2J2000(jd,r)
    rECItoECI(DCM,TEME(), J2000(), jd)*r
end

TEME2J2000 (generic function with 1 method)

In [40]:
tle_str = """
CUBEL                   
1 47448U 21006AM  21104.70471889  .00000294  00000-0  20468-4 0  9997
2 47448  97.5032 167.1437 0011425 327.0810  32.9708 15.12771589 12411
"""
tle = read_tle_from_string(tle_str)[1]

days = 30;

In [43]:
orbp = init_orbit_propagator(Val{:J4}, tle)
T = 24*60*60 / tle.n # Orbital period

_, r, v = propagate!(orbp, 0:T*days); # Probably in TEME frame


t₁ = tle.epoch
t₂ = t₁ + T*days / (24*60*60)

println(r)

# Rotate TEME to J2000
r_j2000 = map(t -> TEME2J2000(t,r), range(t₁, t₂, length=length(1:T*days)))

r_sun = map(t -> sun_position_i(t), range(t₁, t₂, length=length(1:T*days)));

illumination = map(satellite_lighting_condition, r_j2000, r_sun);

StaticArrays.SVector{3, Float64}[[-6.722923820905293e6, 1.536344512932109e6, 14693.788304676238], [-6.722688033794609e6, 1.5373067418953266e6, 22225.129141077352], [-6.722444097857046e6, 1.5382671074374428e6, 29756.443029628703], [-6.722192013404376e6, 1.5392256084014e6, 37287.72084097151], [-6.721931780758287e6, 1.5401822436324283e6, 44818.95344582467], [-6.721663400250371e6, 1.5411370119780183e6, 52350.13171500733], [-6.721386872222114e6, 1.5420899122879442e6, 59881.24651943392], [-6.721102197024923e6, 1.5430409434142474e6, 67412.28873013746], [-6.720809375020084e6, 1.5439901042112515e6, 74943.24921827715], [-6.7205084065788e6, 1.544937393535562e6, 82474.11885515839], [-6.720199292082174e6, 1.5458828102460515e6, 90004.88851221318], [-6.719882031921204e6, 1.5468263532038883e6, 97535.54906105186], [-6.719556626496791e6, 1.5477680212725098e6, 105066.09137345091], [-6.719223076219738e6, 1.5487078133176505e6, 112596.50632136855], [-6.718881381510742e6, 1.5496457282073158e6, 120126.7847769

Excessive output truncated after 524289 bytes.

, -465129.17528308183, 4.822885113789039e6], [4.932492117360777e6, -466992.9101331687, 4.81754520177058e6], [4.937533437679971e6, -468856.08040001546, 4.812199486237289e6], [4.942568810684003e6, -470718.68384192506, 4.8068479736332875e6], [4.94759823031117e6, -472580.7182179008, 4.80149067040959e6], [4.952621690506893e6, -474442.1812876125, 4.796127583024218e6], [4.957639185223784e6, -476303.07081144414, 4.790758717942077e6], [4.9626507084216e6, -478163.3845504547, 4.785384081635029e6], [4.967656254067285e6, -480023.12026641244, 4.780003680581857e6], [4.972655816134975e6, -481882.2757217795, 4.774617521268234e6], [4.977649388605991e6, -483740.8486797269, 4.769225610186747e6], [4.982636965468838e6, -485598.8369041154, 4.763827953836905e6], [4.987618540719257e6, -487456.23815953213, 4.758424558725056e6], [4.992594108360186e6, -489313.0502112682, 4.7530154313644655e6], [4.997563662401769e6, -491169.27082531573, 4.747600578275271e6], [5.00252719686141e6, -493024.8977684034, 4.7421800059844

LoadError: DimensionMismatch("Tried to multiply arrays of size (3, 3) and (171342,)")

In [42]:
illumnication_change = findall(v -> abs(v) > 0, diff(illumination))

_epoch = to_datatime(tle.epoch)
println("EPOCH: ", _epoch)
println("##########")
for c ∈ illumnication_change
    
    ts = _epoch + Dates.Second(c)
    if illumination[c-1] == SAT_LIGHTING_SUNLIGHT
       println("From sunlight to eclipse @ ", ts) 
    end
    if illumination[c-1] == SAT_LIGHTING_UMBRA
       println("From eclipse to sunlight @ ", ts) 
    end
end

EPOCH: 2021-04-14T16:54:48
##########
From eclipse to sunlight @ 2021-04-14T17:06:48
From sunlight to eclipse @ 2021-04-14T18:08:54
From eclipse to sunlight @ 2021-04-14T18:42:04
From sunlight to eclipse @ 2021-04-14T19:44:09
From eclipse to sunlight @ 2021-04-14T20:17:20
From sunlight to eclipse @ 2021-04-14T21:19:24
From eclipse to sunlight @ 2021-04-14T21:52:35
From sunlight to eclipse @ 2021-04-14T22:54:39
From eclipse to sunlight @ 2021-04-14T23:27:51
From sunlight to eclipse @ 2021-04-15T00:29:53
From eclipse to sunlight @ 2021-04-15T01:03:07
From sunlight to eclipse @ 2021-04-15T02:05:08
From eclipse to sunlight @ 2021-04-15T02:38:22
From sunlight to eclipse @ 2021-04-15T03:40:23
From eclipse to sunlight @ 2021-04-15T04:13:38
From sunlight to eclipse @ 2021-04-15T05:15:38
From eclipse to sunlight @ 2021-04-15T05:48:53
From sunlight to eclipse @ 2021-04-15T06:50:53
From eclipse to sunlight @ 2021-04-15T07:24:09
From sunlight to eclipse @ 2021-04-15T08:26:08
From eclipse to sunlig